# Production Workflow with TimeSmith

This notebook demonstrates production-ready features: model serialization, error handling, logging, and best practices for deploying TimeSmith models.

## What You'll Learn

- Model serialization (save/load)
- Error handling with custom exceptions
- Logging configuration
- Data validation
- Production best practices


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import tempfile
from pathlib import Path

from timesmith import (
    SimpleMovingAverageForecaster,
    save_model,
    load_model,
    NotFittedError,
    DataError,
    ValidationError,
)
from timesmith.logging_config import configure_logging
import logging

# Configure logging for production
configure_logging(level="INFO", format_string="detailed")
logger = logging.getLogger(__name__)

np.random.seed(42)
print("Production workflow setup complete!")


## 1. Model Serialization: Save and Load Models

Save fitted models for later use in production.


In [ ]:
# Create and fit a model
dates = pd.date_range("2020-01-01", periods=100, freq="D")
y = pd.Series(np.random.randn(100).cumsum() + 100, index=dates)

forecaster = SimpleMovingAverageForecaster(window=7)
forecaster.fit(y)

print(f"Model fitted: {forecaster.is_fitted}")
print(f"Model parameters: {forecaster.get_params()}")

# Save the model
with tempfile.TemporaryDirectory() as tmpdir:
    model_path = Path(tmpdir) / "production_model.pkl"
    save_model(forecaster, model_path, include_metadata=True)
    
    print(f"\n✓ Model saved to: {model_path}")
    print(f"✓ Metadata saved to: {model_path}.metadata.json")
    
    # Load the model
    loaded_model = load_model(model_path)
    
    print(f"\n✓ Model loaded successfully!")
    print(f"  Type: {type(loaded_model).__name__}")
    print(f"  Is fitted: {loaded_model.is_fitted}")
    print(f"  Parameters match: {loaded_model.get_params() == forecaster.get_params()}")
    
    # Use the loaded model
    forecast = loaded_model.predict(fh=10)
    print(f"\n✓ Forecast generated from loaded model:")
    print(f"  Forecast shape: {forecast.y_pred.shape}")
    print(f"  First 5 values: {forecast.y_pred.head().values}")


## 2. Error Handling: Custom Exceptions

TimeSmith provides informative exceptions for better debugging.


In [ ]:
# Example 1: NotFittedError
forecaster = SimpleMovingAverageForecaster(window=5)

try:
    forecast = forecaster.predict(fh=10)
except NotFittedError as e:
    print("Caught NotFittedError:")
    print(f"  Message: {e.message}")
    print(f"  Context: {e.context}")
    print(f"  Full error: {e}")

# Example 2: DataError
from timesmith.core.data_validation import validate_data_quality

try:
    empty_data = pd.Series([], index=pd.DatetimeIndex([]))
    validate_data_quality(empty_data, min_length=10)
except DataError as e:
    print(f"\nCaught DataError: {e}")
    print(f"  Context: {e.context}")

# Example 3: ValidationError
from timesmith.core.validate import validate_input

try:
    invalid_data = [1, 2, 3]  # Not a Series
    validate_input(invalid_data, scitype="SeriesLike", name="y")
except ValidationError as e:
    print(f"\nCaught ValidationError: {e}")
    print(f"  Context: {e.context}")


## 3. Logging in Production

Configure logging for production monitoring and debugging.


In [ ]:
# Configure logging via environment variable (production approach)
os.environ["TIMESMITH_LOG_LEVEL"] = "DEBUG"
os.environ["TIMESMITH_LOG_FORMAT"] = "detailed"

# Reconfigure (in real app, this would be done at startup)
configure_logging()

# Now operations will log
logger.info("Starting production forecast workflow")
logger.debug("Loading data...")

dates = pd.date_range("2020-01-01", periods=50, freq="D")
y = pd.Series(np.random.randn(50).cumsum() + 100, index=dates)

logger.debug(f"Data loaded: {len(y)} points")
logger.info("Fitting model...")

forecaster = SimpleMovingAverageForecaster(window=7)
forecaster.fit(y)

logger.info("Generating forecast...")
forecast = forecaster.predict(fh=10)

logger.info(f"Forecast complete: {len(forecast.y_pred)} points")
print("\n✓ Logging configured and working!")


## 4. Data Validation for Production

Validate data quality before processing.


In [ ]:
from timesmith.core.data_validation import (
    validate_data_quality,
    validate_forecast_horizon,
    check_data_alignment,
)

# Validate data quality
good_data = pd.Series(np.random.randn(100) + 100, 
                     index=pd.date_range("2020-01-01", periods=100, freq="D"))

try:
    validate_data_quality(good_data, min_length=50)
    print("✓ Data quality check passed")
except DataError as e:
    print(f"✗ Data quality issue: {e}")

# Validate forecast horizon
try:
    fh = validate_forecast_horizon(10, max_horizon=20)
    print(f"✓ Forecast horizon validated: {fh}")
except ValidationError as e:
    print(f"✗ Invalid forecast horizon: {e}")

# Check data alignment
y = good_data
X = pd.DataFrame({"feature": np.random.randn(100)}, index=y.index)

try:
    check_data_alignment(y, X)
    print("✓ Data alignment check passed")
except ValidationError as e:
    print(f"✗ Alignment issue: {e}")


## 5. Complete Production Workflow

Put it all together: load data, validate, fit, save, and deploy.


In [ ]:
def production_forecast_workflow(data: pd.Series, forecast_horizon: int, model_path: Path):
    """Complete production workflow with error handling and logging."""
    logger.info("=" * 60)
    logger.info("Starting production forecast workflow")
    logger.info("=" * 60)
    
    try:
        # 1. Validate data
        logger.info("Step 1: Validating input data")
        validate_data_quality(data, min_length=20)
        validate_forecast_horizon(forecast_horizon, max_horizon=100)
        logger.info("✓ Data validation passed")
        
        # 2. Create and fit model
        logger.info("Step 2: Creating and fitting model")
        forecaster = SimpleMovingAverageForecaster(window=7)
        forecaster.fit(data)
        logger.info("✓ Model fitted successfully")
        
        # 3. Generate forecast
        logger.info("Step 3: Generating forecast")
        forecast = forecaster.predict(fh=forecast_horizon)
        logger.info(f"✓ Forecast generated: {len(forecast.y_pred)} points")
        
        # 4. Save model
        logger.info("Step 4: Saving model for production")
        save_model(forecaster, model_path, include_metadata=True)
        logger.info(f"✓ Model saved to: {model_path}")
        
        return forecast, forecaster
        
    except (DataError, ValidationError, NotFittedError) as e:
        logger.error(f"Workflow failed: {e}")
        logger.error(f"Context: {e.context}")
        raise
    except Exception as e:
        logger.error(f"Unexpected error: {e}")
        raise

# Run the workflow
dates = pd.date_range("2020-01-01", periods=80, freq="D")
production_data = pd.Series(np.random.randn(80).cumsum() + 100, index=dates)

with tempfile.TemporaryDirectory() as tmpdir:
    model_path = Path(tmpdir) / "production_model.pkl"
    
    forecast, model = production_forecast_workflow(
        production_data, 
        forecast_horizon=14,
        model_path=model_path
    )
    
    print("\n" + "=" * 60)
    print("Production Workflow Complete!")
    print("=" * 60)
    print(f"\nForecast Summary:")
    print(f"  Points: {len(forecast.y_pred)}")
    print(f"  Mean: {forecast.y_pred.mean():.2f}")
    print(f"  Std: {forecast.y_pred.std():.2f}")
    print(f"\nModel saved and ready for deployment!")
